In [1]:
import pandas as pd
from rapidfuzz import process, fuzz
from tqdm import tqdm
from joblib import Parallel, delayed

seed_value = 42

In [2]:
spotify_df = pd.read_csv('/work/data/songs_with_attributes_and_lyrics.csv')
hot100_df = pd.read_csv('/work/data/hot100.csv')

In [3]:
spotify_df = spotify_df.sample(frac=0.05, random_state=seed_value)
hot100_df = hot100_df.sample(frac=0.05, random_state=seed_value)

### Trying to match up `spotify_df` to `hot100_df`

In [6]:
# Cleaning up artists and song names
spotify_df['spotify_artist_clean'] = spotify_df['artists'].str.lower().str.strip()
spotify_df['spotify_name_clean'] = spotify_df['name'].str.lower().str.strip()
hot100_df['hot100_artist_clean'] = hot100_df['Artist'].str.lower().str.strip()
hot100_df['hot100_name_clean'] = hot100_df['Song'].str.lower().str.strip()

# Merging on both song and artist
spotify_labeled = spotify_df.merge(
    hot100_df[['hot100_name_clean', 'hot100_artist_clean']].drop_duplicates().assign(is_hit=1),
    how='left',
    left_on=['spotify_name_clean', 'spotify_artist_clean'],
    right_on=['hot100_name_clean', 'hot100_artist_clean']
)

# Labeling hits
spotify_labeled['is_hit'] = spotify_labeled['is_hit'].fillna(0).astype(int)

# Building the output 
output_df = spotify_labeled[[
    'name', 'artists', 'spotify_name_clean', 'spotify_artist_clean', 'is_hit'
]].copy()
output_df.rename(columns={
    'name': 'spotify_original_song',
    'artists': 'spotify_original_artist',
}, inplace=True)

# Merging again to bring over the Hot 100 original values 
output_df = output_df.merge(
    hot100_df[['hot100_name_clean', 'hot100_artist_clean', 'Song', 'Artist']],
    left_on=['spotify_name_clean', 'spotify_artist_clean'],
    right_on=['hot100_name_clean', 'hot100_artist_clean'],
    how='left')
output_df.rename(columns={
    'Song': 'hot100_original_song',
    'Artist': 'hot100_original_artist',
}, inplace=True)

# Output to csv for checks
output_df.to_csv('spotify_matched_and_unmatched_songs.csv', index=False)
print(output_df['is_hit'].value_counts())
output_df[output_df['is_hit'] == 1]

is_hit
0    47566
1      338
Name: count, dtype: int64


,spotify_original_song,spotify_original_artist,spotify_name_clean,spotify_artist_clean,is_hit,hot100_name_clean,hot100_artist_clean,hot100_original_song,hot100_original_artist
674,The Wild Boys,Duran Duran,the wild boys,duran duran,1,the wild boys,duran duran,The Wild Boys,Duran Duran
884,Stitches,Shawn Mendes,stitches,shawn mendes,1,stitches,shawn mendes,Stitches,Shawn Mendes
885,Stitches,Shawn Mendes,stitches,shawn mendes,1,stitches,shawn mendes,Stitches,Shawn Mendes
1012,Semi-Charmed Life,Third Eye Blind,semi-charmed life,third eye blind,1,semi-charmed life,third eye blind,Semi-Charmed Life,Third Eye Blind
1013,Semi-Charmed Life,Third Eye Blind,semi-charmed life,third eye blind,1,semi-charmed life,third eye blind,Semi-Charmed Life,Third Eye Blind
...,...,...,...,...,...,...,...,...,...
47322,Send My Love (To Your New Lover),Adele,send my love (to your new lover),adele,1,send my love (to your new lover),adele,Send My Love (To Your New Lover),Adele
47323,Send My Love (To Your New Lover),Adele,send my love (to your new lover),adele,1,send my love (to your new lover),adele,Send My Love (To Your New Lover),Adele
47668,Wonderful Christmastime,Paul McCartney,wonderful christmastime,paul mccartney,1,wonderful christmastime,paul mccartney,Wonderful Christmastime,Paul McCartney
47669,Wonderful Christmastime,Paul McCartney,wonderful christmastime,paul mccartney,1,wonderful christmastime,paul mccartney,Wonderful Christmastime,Paul McCartney


<hr>

In [8]:


# -----------------------------------
# 1. Cleaning up artists and song names
# -----------------------------------
spotify_df['spotify_artist_clean'] = spotify_df['artists'].str.lower().str.strip()
spotify_df['spotify_name_clean'] = spotify_df['name'].str.lower().str.strip()
hot100_df['hot100_artist_clean'] = hot100_df['Artist'].str.lower().str.strip()
hot100_df['hot100_name_clean'] = hot100_df['Song'].str.lower().str.strip()

# -----------------------------------
# 2. Initial exact match
# -----------------------------------
spotify_labeled = spotify_df.merge(
    hot100_df[['hot100_name_clean', 'hot100_artist_clean']].drop_duplicates().assign(is_hit=1),
    how='left',
    left_on=['spotify_name_clean', 'spotify_artist_clean'],
    right_on=['hot100_name_clean', 'hot100_artist_clean']
)

# Labeling hits
spotify_labeled['is_hit'] = spotify_labeled['is_hit'].fillna(0).astype(int)

# Separate matched and unmatched
matched_df = spotify_labeled[spotify_labeled['is_hit'] == 1].copy()
unmatched_df = spotify_labeled[spotify_labeled['is_hit'] == 0].copy()

print(f"Exact matches found: {len(matched_df)}")
print(f"Unmatched songs needing fuzzy matching: {len(unmatched_df)}")

# -----------------------------------
# 3. Prepare for fuzzy matching
# -----------------------------------
hot100_choices = list(zip(hot100_df['hot100_name_clean'], hot100_df['hot100_artist_clean']))
hot100_combined = [f"{song} - {artist}" for song, artist in hot100_choices]

# -----------------------------------
# 4. Define fuzzy match function
# -----------------------------------
def fuzzy_match_row(row, choices, threshold=90):
    query = f"{row['spotify_name_clean']} - {row['spotify_artist_clean']}"
    match = process.extractOne(
        query,
        choices,
        scorer=fuzz.token_sort_ratio,
        score_cutoff=threshold
    )
    if match:
        best_match, score, _ = match
        if " - " in best_match:
            best_name, best_artist = best_match.split(' - ', 1)
        else:
            best_name, best_artist = best_match, ""
        return {
            'hot100_name_clean': best_name.strip(),
            'hot100_artist_clean': best_artist.strip(),
            'fuzzy_score': score
        }
    else:
        return {
            'hot100_name_clean': None,
            'hot100_artist_clean': None,
            'fuzzy_score': None
        }

# -----------------------------------
# 5. Apply fuzzy matching in parallel
# -----------------------------------
def parallel_fuzzy_match(df, choices, n_jobs=-1):
    results = Parallel(n_jobs=n_jobs, backend="loky")(
        delayed(fuzzy_match_row)(row, choices) for _, row in tqdm(df.iterrows(), total=len(df), desc="Fuzzy matching songs in parallel...")
    )
    return pd.DataFrame(results)

# Run parallel fuzzy matching
fuzzy_matches = parallel_fuzzy_match(unmatched_df, hot100_combined)

# Drop any existing columns from unmatched_df that fuzzy_matches will add
columns_to_drop = ['hot100_name_clean', 'hot100_artist_clean', 'fuzzy_score']
unmatched_df = unmatched_df.drop(columns=[col for col in columns_to_drop if col in unmatched_df.columns])

# Now safe to concat
unmatched_df = pd.concat([unmatched_df.reset_index(drop=True), fuzzy_matches.reset_index(drop=True)], axis=1)


# Filter successful fuzzy matches
fuzzy_matched = unmatched_df[unmatched_df['hot100_name_clean'].notnull()].copy()
fuzzy_matched['is_hit'] = 1

# -----------------------------------
# 6. Combine all matches
# -----------------------------------
final_matched = pd.concat([matched_df, fuzzy_matched], ignore_index=True)

# Building the output
output_df = final_matched[[
    'name', 'artists', 'spotify_name_clean', 'spotify_artist_clean', 'hot100_name_clean', 'hot100_artist_clean', 'is_hit'
]].copy()

output_df.rename(columns={
    'name': 'spotify_original_song',
    'artists': 'spotify_original_artist',
}, inplace=True)

# Merge Hot100 original info back
output_df = output_df.merge(
    hot100_df[['hot100_name_clean', 'hot100_artist_clean', 'Song', 'Artist']],
    on=['hot100_name_clean', 'hot100_artist_clean'],
    how='left'
)

output_df.rename(columns={
    'Song': 'hot100_original_song',
    'Artist': 'hot100_original_artist',
}, inplace=True)

# -----------------------------------
# 7. Output results
# -----------------------------------
output_df.to_csv('spotify_matched_and_unmatched_songs_with_fuzzy.csv', index=False)

# -----------------------------------
# 8. Summary
# -----------------------------------
print("\n🎯 Matching summary:")
print(output_df['is_hit'].value_counts())

print("\n🆕 Songs newly matched via fuzzy matching:")
fuzzy_matched[['name', 'artists', 'spotify_name_clean', 'spotify_artist_clean', 'fuzzy_score']]

Exact matches found: 200
Unmatched songs needing fuzzy matching: 47566
Fuzzy matching songs in parallel...:  36%|███▋      | 17290/47566 [02:57<05:58, 84.36it/s]

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=068aa808-4695-4d75-a14b-785a94b3cf38' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>